## ref : [***STGCN튜토리얼***](https://miruetoto.github.io/yechan3/posts/3_Researches/ITSTGCN/2022-12-29-STGCN-tutorial.html)

In [1]:
# !pip install torch-geometric
# !pip install torch-geometric-temporal

## import

`-` 필요한 패키지 임포트

In [2]:
# 일반적인 모듈 
import numpy as np
import matplotlib.pyplot as plt 
import networkx as nx 
from tqdm import tqdm 

# 파이토치 관련 
import torch
import torch.nn.functional as F

# PyG 관련 
from torch_geometric.data import Data

# STGCN 관련 
import torch_geometric_temporal
from torch_geometric_temporal.nn.recurrent import GConvGRU
from torch_geometric_temporal.signal import temporal_signal_split 

- `tqdm`: for문의 진행상태를 확인하기 위한 패키지
- `networkx`: 그래프 시그널 시각화를 위한 모듈
- `torch` : 파이토치 (STGCN은 파이토치 기반으로 만들어짐) 모듈
- `torch.nn.functional`: relu 등의 활성화함수를 불러오기 위한 모듈
- `Data` : 그래프 자료를 만들기 위한 클래스
- `GConvGRU` : STGCN layer를 만드는 클래스
- `temporal_signal_split` : STGCN dataset을 train/test 형태로 분리하는 기능이 있는 "함수"

`-` STGCN 학습을 위한 클래스 선언

In [3]:
class RecurrentGCN(torch.nn.Module):
    def __init__(self, node_features, filters):
        super(RecurrentGCN, self).__init__()
        self.recurrent = GConvGRU(node_features, filters, 2)
        self.linear = torch.nn.Linear(filters, 1)

    def forward(self, x, edge_index, edge_weight):
        h = self.recurrent(x, edge_index, edge_weight)
        h = F.relu(h)
        h = self.linear(h)
        return h

## 데이터셋 만들기

### `node id` > edges > weight > feature

- ${\bf f}=\begin{bmatrix} {\bf f}_1\\ {\bf f}_2\\ \dots \\ {\bf f}_{521} \end{bmatrix}=\begin{bmatrix} f(t=1,v=\tt{BACS}) & \dots & f(t=1,v=\tt{ZALA}) \\ f(t=2,v=\tt{BACS}) & \dots & f(t=2,v=\tt{ZALA}) \\ \dots & \dots & \dots \\ f(t=521,v=\tt{BACS}) & \dots & f(t=521,v=\tt{ZALA}) \end{bmatrix}$